In [ ]:
import torch
import torchvision
import torch.utils.data as data
import torchvision.transforms as transforms

from torchinfo import summary

import numpy as np
import time

In [ ]:
IMAGENET_DIR = '/ImageNet/'
BATCH_SIZE = 1
LOADER_WORKERS = 0

In [ ]:
model_file = '../Flask/Models/mobilenet_v2-script.pt'

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
model = torch.jit.load(model_file)
model = model.to(device)
model = model.eval()

In [ ]:
# summary(model, (BATCH_SIZE, 3, 224, 224))

In [ ]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
dataset = torchvision.datasets.ImageNet(root=IMAGENET_DIR, transform=transform, split='val')
loader = data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=LOADER_WORKERS, pin_memory=True)

In [ ]:
n_top1 = 0
n_top5 = 0
cnt = 0

s = time.time()
pred_tms = []
with torch.no_grad():
    for images, labels in loader:

        x = images.to(device)
        y = labels.to(device)
        
        ss = time.time()
        output = model(x)
        pred_tms.append(time.time()-ss)
        
        cnt += output.size(0)
        
        _, pred_top1 = output.max(1)
        _, pred_top5 = output.topk(5, 1, True, True)
        
        n_top1 += torch.eq(pred_top1, y).sum().item()
        n_top5 += torch.isin(pred_top5, y).max(1)[0].sum().item()
        
        print(f"\rstep: {cnt}/{len(dataset)}", end='')
        
total_tm = time.time() - s
print()
print(f"top-1:  {n_top1/cnt:0.4f}")
print(f"top-5:  {n_top5/cnt:0.4f}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Total Time: {total_tm:0.4f} ({total_tm/len(dataset):0.4f})")
print(f"Average Prediction Time: {np.mean(pred_tms):0.4f}")